In [ ]:
import os
import numpy as np

import pandas as pd
import tensorflow as tf

from transformers import BertTokenizer
from model import BertForQALabeling
from preprocessing import dataPreprocessor
from parameters import *

from sklearn.model_selection import KFold

from scipy.stats import spearmanr

import horovod.tensorflow as hvd

In [ ]:
hvd.init()

gpus = tf.config.list_physical_devices('GPU') 
for gpu in gpus:
    print(gpu)
    tf.config.experimental.set_memory_growth(gpu, True)
if gpus:
    print("gpus ", gpus)
    print("local rank ",hvd.local_rank())
    tf.config.set_visible_devices(gpus[hvd.local_rank()], 'GPU')
    print(tf.config.get_visible_devices())

In [ ]:
data_dir = "google-quest-challenge/"

train_df = pd.read_csv(os.path.join(data_dir, "train.csv"))
test_df = pd.read_csv(os.path.join(data_dir, "test.csv"))
submit_df = pd.read_csv(os.path.join(data_dir, "sample_submission.csv"))
stack_df = pd.read_csv(os.path.join(data_dir, "stackexchange.csv"))
stack_df.head()

In [ ]:
train_X_df = train_df[['question_title', 'question_body', 'answer']]
train_targets_df = train_df[["question_asker_intent_understanding",
    "question_body_critical",
    "question_conversational",
    "question_expect_short_answer",
    "question_fact_seeking",
    "question_has_commonly_accepted_answer",
    "question_interestingness_others",
    "question_interestingness_self",
    "question_multi_intent",
    "question_not_really_a_question",
    "question_opinion_seeking",
    "question_type_choice",
    "question_type_compare",
    "question_type_consequence",
    "question_type_definition",
    "question_type_entity",
    "question_type_instructions",
    "question_type_procedure",
    "question_type_reason_explanation",
    "question_type_spelling",
    "question_well_written",
    "answer_helpful",
    "answer_level_of_information",
    "answer_plausible",
    "answer_relevance",
    "answer_satisfaction",
    "answer_type_instructions",
    "answer_type_procedure",
    "answer_type_reason_explanation",
    "answer_well_written"]]

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
q_title = train_X_df['question_title'].values
q_body = train_X_df['question_body'].values
answer = train_X_df['answer'].values

targets = train_targets_df.to_numpy()

In [ ]:
dataPreprocessor.logger = False
dataPreprocessor.tokenizer = tokenizer

In [8]:
preprocessedInput = dataPreprocessor.preprocessBatch(q_body, q_title, answer, max_seq_lengths=(26,260,210,500))

Token indices sequence length is longer than the specified maximum sequence length for this model (645 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2491 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (855 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (977 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (557 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for th

In [9]:
print(preprocessedInput.shape)
print(targets.shape)

print(preprocessedStack.shape)

(6079, 3, 500)
(6079, 30)


In [10]:
def accumulated_gradients(gradients,
                          step_gradients,
                          num_grad_accumulates) -> tf.Tensor:
    if gradients is None:
        gradients = [flat_gradients(g) / num_grad_accumulates for g in step_gradients]
    else:
        for i, g in enumerate(step_gradients):
            gradients[i] += flat_gradients(g) / num_grad_accumulates
    
    return gradients

# This is needed for tf.gather like operations.
def flat_gradients(grads_or_idx_slices: tf.Tensor) -> tf.Tensor:
    '''Convert gradients if it's tf.IndexedSlices.
    When computing gradients for operation concerning `tf.gather`, the type of gradients 
    '''
    if type(grads_or_idx_slices) == tf.IndexedSlices:
        return tf.scatter_nd(
            tf.expand_dims(grads_or_idx_slices.indices, 1),
            grads_or_idx_slices.values,
            grads_or_idx_slices.dense_shape
        )
    return grads_or_idx_slices

In [11]:
def compute_spearmanr_ignore_nan(trues, preds):
    print("trues = ", trues.shape)
    print("preds = ", preds.shape)
    rhos = []
    for tcol, pcol in zip(np.transpose(trues), np.transpose(preds)):
        rhos.append(spearmanr(tcol, pcol).correlation)
    return np.nanmean(rhos)

def spearman_metric(y_true, y_pred):
    corr = [
        spearmanr(pred_col, target_col).correlation
        for pred_col, target_col in zip(y_pred.T, y_true.T)
    ]
    return corr

def calculate_sper(true, preds):
    score = 0
    for i in range(30):
        score += np.nan_to_num(spearmanr(true[:, i], preds[:, i]).correlation)
    return score/30

In [12]:
a = np.array([1, 2, 3])
print(a.shape)
b = np.array([[2], [3], [4]])
print(b.shape)
print("v stack ", np.vstack(b).shape)
print(np.transpose(b).shape)

(3,)
(3, 1)
v stack  (3, 1)
(1, 3)


In [13]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, warmup_steps, num_steps, base_lr):
    super(CustomSchedule, self).__init__()

    self.warmup_steps = tf.cast(warmup_steps, tf.float32)
    self.num_steps = tf.cast(num_steps, tf.float32)
    self.lr = tf.cast(base_lr, tf.float32)

  def __call__(self, step):
    def warmupPhase() : return step/tf.math.maximum(1.0, self.warmup_steps)
    def decayPhase() : return tf.math.maximum(0.0, (self.num_steps - step))/tf.math.maximum(1.0, self.num_steps - self.warmup_steps)

    multiplier = tf.cond(tf.math.less(step, self.warmup_steps), warmupPhase, decayPhase)
    
    return self.lr * multiplier

kf = KFold(n_splits=5)
fold_nr =0

for train_idx, test_idx in kf.split(preprocessedInput):
    print("                FOLD ", fold_nr)

    # train test indices
    train_input = tf.gather(preprocessedInput, train_idx, axis=0)
    train_target = tf.gather(targets, train_idx, axis=0)

    test_input = tf.gather(preprocessedInput, test_idx, axis=0)
    test_target = tf.gather(targets, test_idx, axis=0)

    #train dataset
    train_ds = tf.data.Dataset.from_tensor_slices((train_input, train_target)). \
                             shuffle(len(train_input)//4, reshuffle_each_iteration=True). \
                             batch(batch_size=batch_size, drop_remainder=True)

    #test dataset
    test_ds = tf.data.Dataset.from_tensor_slices((test_input, test_target)). \
                             shuffle(len(test_input)//4, reshuffle_each_iteration=True). \
                             batch(batch_size=batch_size, drop_remainder=True)

    lr_scheduler = CustomSchedule(warmup_steps=warmup_steps*2, 
                                  num_steps=targets.shape[0]//batch_size, 
                                  base_lr=lr*hvd.size())

    #optimizer = tfa.optimizers.AdamW(learning_rate=lr_scheduler, weight_decay=decay)
    optimizer = tf.optimizers.Adam(learning_rate = lr_scheduler)
    bce_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

    model = BertForQALabeling.from_pretrained('bert-base-uncased', num_labels=num_labels, output_hidden_states=True)
    model.backbone.bert.pooler._trainable=False
    trainable = model.trainable_variables

    checkpoint_dir = './checkpoints/best_best_uncased_fold_{}' .format(fold_nr)
    checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer)
    

    @tf.function
    def train_step(inputs, y_true, first_batch):
        with tf.GradientTape() as tape:
            ids_mask, type_ids_mask, attention_mask = inputs[:, 0, :], inputs[:, 1, :], inputs[:, 2, :]
            y_pred = model(ids_mask, 
                             attention_mask= attention_mask, 
                             token_type_ids=type_ids_mask, 
                             training=True)
            loss = tf.reduce_sum(bce_loss(y_true, y_pred)*(1. / batch_size))

        tape = hvd.DistributedGradientTape(tape)

        grads = tape.gradient(loss, trainable)

        return loss, grads, tf.math.sigmoid(y_pred)

    @tf.function
    def test_step(inputs, y_true):
        ids_mask, type_ids_mask, attention_mask = inputs[:, 0, :], inputs[:, 1, :], inputs[:, 2, :]
        y_pred = model(ids_mask, 
                      attention_mask= attention_mask, 
                      token_type_ids=type_ids_mask, 
                      training=False)
        loss = tf.reduce_sum(bce_loss(y_true, y_pred)*(1. / batch_size))
        
        return loss, tf.math.sigmoid(y_pred)

    last_loss = 999999

    for epoch in range(epochs):
        gradients = None
        train_losses = []
        test_losses = []
        train_preds = []
        test_preds = []
        train_targets = []
        test_targets = []
        global_batch = 0
        for batch_nr, (inputs, y_true) in enumerate(train_ds):
            loss, current_gradient, y_pred = train_step(inputs, y_true, batch_nr==0)
            train_losses.append(np.mean(loss))
            train_preds.append(y_pred)
            train_targets.append(y_true)
            gradients = accumulated_gradients(gradients, current_gradient, gradient_accumulate_steps)

            if (batch_nr +1)%gradient_accumulate_steps ==0:
                #print("batch_nr {} gradient applying" .format(batch_nr))
                optimizer.apply_gradients(zip(gradients, trainable))
                global_batch +=1
                gradients = None

                if batch_nr == 0:
                    print("first batch")
                    hvd.broadcast_variables(trainable, root_rank=0)
                    hvd.broadcast_variables(optimizer.variables(), root_rank=0)

            if batch_nr % 100 == 0 and hvd.local_rank() == 0:
                print('Step {} loss {}'  .format(batch_nr, loss, ))

        for _, (inputs, y_true) in enumerate(test_ds):
            loss, y_pred = test_step(inputs, y_true)
            test_losses.append(np.mean(loss))
            test_preds.append(y_pred)
            test_targets.append(y_true)

        test_spearmans = spearman_metric(np.vstack(test_targets), np.vstack(test_preds))
        train_spearmans = spearman_metric(np.vstack(train_targets), np.vstack(train_preds))
        
        print("epoch {} train loss {} test loss {} test spearman metric {}%6 train spearman metric {}%6" \
              .format(epoch, np.mean(train_losses), np.mean(test_losses), np.mean(test_spearmans), np.mean(train_spearmans)))
        
        if np.mean(test_spearmans) < last_loss:
            if hvd.rank() == 0:
                checkpoint.save(os.path.join(checkpoint_dir, "best_base_uncased_best_model"))
                last_loss = np.mean(test_spearmans)
                print("saving checkpoint... ")
    
    """    
        Pseudo labeling for given fold using stackexchange data
        
            1. restoring best checkpoint for given fold
            2. predicting output values for stackexchange
            3. saving predicted data into csv file 
    """
    checkpoint.restore(checkpoint_dir).assert_consumed()
    pseudo_labeling_ds = tf.data.Dataset.from_tensor_slices((preprocessedStack)).batch(batch_size=batch_size, drop_remainder=True)
    pseudo_predictions = []
    for _, inputs in enumerate(pseudo_labeling_ds):
        ids_mask, type_ids_mask, attention_mask = inputs[:, 0, :], inputs[:, 1, :], inputs[:, 2, :]
        predicted = model(ids_mask, 
                      attention_mask= attention_mask, 
                      token_type_ids=type_ids_mask, 
                      training=False)
        
        predicted = tf.math.sigmoid(predicted)

        pseudo_predictions.extend(predicted.numpy())
    
    predicted_df = stack_df.copy()
    for idx, col in enumerate(train_targets_df.columns):
        predicted_df[col] = pseudo_predictions[:, idx]
    
    predicted_df.to_csv(
        os.path.join('./dataframes/pseudo_labeled' "best_base_uncased_fold-{}.csv" .format(fold_nr)), index=False)
    
    fold_nr += 1

                FOLD  0
False





<class 'list'>
bert_for_qa_labeling/bert_backbone/bert/embeddings/word_embeddings/weight:0
True
bert_for_qa_labeling/bert_backbone/bert/embeddings/position_embeddings/embeddings:0
True
bert_for_qa_labeling/bert_backbone/bert/embeddings/token_type_embeddings/embeddings:0
True
bert_for_qa_labeling/bert_backbone/bert/embeddings/LayerNorm/gamma:0
True
bert_for_qa_labeling/bert_backbone/bert/embeddings/LayerNorm/beta:0
True
bert_for_qa_labeling/bert_backbone/bert/encoder/layer_._0/attention/self/query/kernel:0
True
bert_for_qa_labeling/bert_backbone/bert/encoder/layer_._0/attention/self/query/bias:0
True
bert_for_qa_labeling/bert_backbone/bert/encoder/layer_._0/attention/self/key/kernel:0
True
bert_for_qa_labeling/bert_backbone/bert/encoder/layer_._0/attention/self/key/bias:0
True
bert_for_qa_labeling/bert_backbone/bert/encoder/layer_._0/attention/self/value/kernel:0
True
bert_for_qa_labeling/bert_backbone/bert/encoder/layer_._0/attention/self/value/bias:0

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 3319, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-b313336bcdd6>", line 101, in <module>
    loss, current_gradient, y_pred = train_step(inputs, y_true, batch_nr==0)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/eager/def_function.py", line 568, in __call__
    result = self._call(*args, **kwds)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/eager/def_function.py", line 606, in _call
    results = self._stateful_fn(*args, **kwds)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/eager/function.py", line 2363, in __call__
    return graph_function._filtered_call(args, kwargs)  # pylint: disable=protected-access
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/eager/function.py", line 1611, in _filtered_call
    self.captured_inputs)
 

KeyboardInterrupt: 